In [ ]:
!pip install duckdb duckdb-engine s3fs

In [2]:
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from faker import Faker
import random
import os
import pandas as pd

In [4]:
# Conectando ao MinIO via S3 (ajustar se usar local)
con = duckdb.connect(database=':memory:')

# Conectar ao Parquet no MinIO
# s3_path = "s3://bronze/transacoes_duck_bilhao.parquet"

# S3 config para DuckDB
con.execute("""
SET s3_endpoint='localhost:9000';
SET s3_region='us-east-1';
SET s3_access_key_id='admin';
SET s3_secret_access_key='password123';
SET s3_url_style='path';
SET s3_use_ssl=false;
""")

# Carrega Parquet
#df = con.execute(f"SELECT * FROM '{s3_path}'").fetchdf()
df = con.execute("SELECT * FROM 'transacoes_duck_bilhao.parquet'").fetchdf()

# ✅ Análises básicas
print("✔️ Contagem de linhas:", len(df))
print("\n✔️ Tipos das colunas:\n", df.dtypes)
print("\n✔️ Estatísticas descritivas:\n", df.describe())


✔️ Contagem de linhas: 1000000

✔️ Tipos das colunas:
 transaction_id                 int64
customer_id                    int64
transaction_date      datetime64[us]
transaction_amount           float64
merchant_category             object
payment_method                object
dtype: object

✔️ Estatísticas descritivas:
        transaction_id   customer_id            transaction_date  \
count  1000000.000000  1.000000e+06                     1000000   
mean    500000.500000  4.999361e+08  2024-07-21 16:28:11.481601   
min          1.000000  2.302000e+03         2023-07-23 00:00:00   
25%     250000.750000  2.499371e+08         2024-01-20 00:00:00   
50%     500000.500000  4.997303e+08         2024-07-21 00:00:00   
75%     750000.250000  7.498587e+08         2025-01-21 00:00:00   
max    1000000.000000  9.999963e+08         2025-07-22 00:00:00   
std     288675.278932  2.885898e+08                         NaN   

       transaction_amount  
count      1000000.000000  
mean          2504